# SQL Operation in IMDB

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

In [3]:
cursor = conn.cursor()

## Data Manipulation

In [4]:
cursor.execute('UPDATE Movie SET year = REPLACE(year, "X ", "");')

cursor.execute('UPDATE Movie SET year = REPLACE(year, "V", "");')

cursor.execute('UPDATE Movie SET year = REPLACE(year, "I", "");')

cursor.execute('UPDATE Movie SET title = LTRIM(title);')
cursor.execute('UPDATE Movie SET year = RTRIM(LTRIM(year));')
cursor.execute('UPDATE Movie SET rating = RTRIM(LTRIM(rating));')
cursor.execute('UPDATE Movie SET num_votes = RTRIM(LTRIM(num_votes));')

cursor.execute('UPDATE M_Producer SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Producer SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Director SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Director SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Cast SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE M_Cast SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE M_Genre SET gid = RTRIM(LTRIM(gid));')
cursor.execute('UPDATE M_Genre SET mid = RTRIM(LTRIM(mid));')

cursor.execute('UPDATE Genre SET gid = RTRIM(LTRIM(gid));')
cursor.execute('UPDATE Genre SET name = RTRIM(LTRIM(name));')

cursor.execute('UPDATE Person SET name = RTRIM(LTRIM(name));')
cursor.execute('UPDATE Person SET pid = RTRIM(LTRIM(pid));')
cursor.execute('UPDATE Person SET gender = RTRIM(LTRIM(gender));')

conn.commit()


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [6]:
%%time
# Write your sql query below

query = """
       SELECT p.Name as Director,m.title as Movie_Name,g.Name as Genre,m.year as Leap_Year
FROM movie m JOIN M_Director d ON m.MID=d.MID
JOIN M_Genre mg ON m.MID=mg.MID
JOIN Person p ON d.PID=p.PID
JOIN Genre g ON mg.GID=g.GID
WHERE (m.year%4=0) AND g.Name LIKE "%%Comedy%%"
        
        """

q1 = pd.read_sql_query(query, conn)    
print(q1.shape)
q1.head()

(232, 4)
Wall time: 149 ms


,Director,Movie_Name,Genre,Leap_Year
0,Milap Zaveri,Mastizaade,Comedy,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,"Adventure, Comedy",2004
2,Anurag Kashyap,Gangs of Wasseypur,"Action, Comedy, Crime",2012
3,Frank Coraci,Around the World in 80 Days,"Action, Adventure, Comedy",2004
4,Griffin Dunne,The Accidental Husband,"Comedy, Romance",2008


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [7]:
%%time
# Write your sql query below

query = """
        SELECT m.title as Movie_Name,p.Name AS Actor_Name
FROM M_Cast mc INNER JOIN Person p ON mc.PID=p.PID
INNER JOIN Movie m ON mc.MID=m.MID
WHERE m.title="Anand"
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
print(q2)

(17, 2)
   Movie_Name        Actor_Name
0       Anand  Amitabh Bachchan
1       Anand     Rajesh Khanna
2       Anand    Brahm Bhardwaj
3       Anand        Ramesh Deo
4       Anand         Seema Deo
5       Anand        Dev Kishan
6       Anand       Durga Khote
7       Anand     Lalita Kumari
8       Anand      Lalita Pawar
9       Anand      Atam Prakash
10      Anand     Sumita Sanyal
11      Anand    Asit Kumar Sen
12      Anand        Dara Singh
13      Anand     Johnny Walker
14      Anand         Moolchand
15      Anand      Gurnam Singh
16      Anand            Savita
Wall time: 348 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [8]:
%%time
# Write your sql query below

query = """

SELECT DISTINCT p.PID,p.Name FROM
Person p INNER JOIN M_Cast mc ON p.PID=mc.PID
INNER JOIN Movie m ON mc.MID=m.MID
WHERE m.year<1970 AND p.PID IN  
		(SELECT DISTINCT p.PID
		FROM Person p INNER JOIN M_Cast mc ON p.PID=mc.PID
		INNER JOIN Movie m ON mc.MID=m.MID
		WHERE m.year>1990) order by p.PID
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
#q3.head()
print(q3)

(300, 2)
           PID                Name
0    nm0000821    Amitabh Bachchan
1    nm0003987  Mohandas K. Gandhi
2    nm0004334               Rekha
3    nm0004429          Dharmendra
4    nm0004432   Prithviraj Kapoor
..         ...                 ...
295  nm2983292         Mohamad Ali
296  nm4563339                Dube
297  nm7390393      Aachi Manorama
298  nm9036653       Surendra Rahi
299  nm9045541             Shirley

[300 rows x 2 columns]
Wall time: 1.34 s


### trying different way 

In [9]:
%%time
# Write your sql query below

query = """

SELECT DISTINCT p1.PID,p1.Name 
FROM Person p1 
INNER JOIN M_Cast c1 ON p1.PID = c1.PID
INNER JOIN Movie m1 ON m1.MID = c1.MID,

Person p2
INNER JOIN M_Cast c2 ON p2.PID = c2.PID
INNER JOIN Movie m2 ON m2.MID = c2.MID

WHERE m1.year < 1970 and m2.year > 1990 and c1.PID = c2.PID ORDER BY p1.PID
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
#q3.head()
print(q3)

(300, 2)
           PID                Name
0    nm0000821    Amitabh Bachchan
1    nm0003987  Mohandas K. Gandhi
2    nm0004334               Rekha
3    nm0004429          Dharmendra
4    nm0004432   Prithviraj Kapoor
..         ...                 ...
295  nm2983292         Mohamad Ali
296  nm4563339                Dube
297  nm7390393      Aachi Manorama
298  nm9036653       Surendra Rahi
299  nm9045541             Shirley

[300 rows x 2 columns]
Wall time: 1.41 s


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [11]:
%%time
# Write your sql query below

query = """
SELECT p.name AS Dir_Name, COUNT(*) movie_count
FROM M_Director d 
     INNER JOIN Person p 
              ON p.PID = d.PID
     INNER JOIN Movie m
              ON m.MID = d.MID              
GROUP BY p.name HAVING movie_count>10 ORDER BY movie_count DESC LIMIT 5
        
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
#q4.head()
print(q4)

(5, 2)
          Dir_Name  movie_count
0     David Dhawan           39
1     Mahesh Bhatt           36
2  Ram Gopal Varma           30
3     Priyadarshan           30
4     Vikram Bhatt           29
Wall time: 106 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [12]:
%%time
# Write your sql query below

query = """
  
SELECT m.title,m.year,COUNT(*) AS fmovie_count
FROM Movie m
WHERE m.MID NOT IN (
  SELECT DISTINCT m.MID
  FROM Movie m
  INNER JOIN M_Cast mc ON m.MID = mc.MID
  INNER JOIN Person p  ON mc.PID = p.PID
  AND p.gender = 'Male'
)
AND m.MID IN (
  SELECT DISTINCT m.MID
  FROM Movie m
  INNER JOIN M_Cast mc ON m.MID = mc.MID
  INNER JOIN Person p  ON mc.PID = p.PID
  AND p.gender = 'Female'
)
GROUP BY m.year;

        
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
#q5a.head(6)
print(q5a)

(4, 3)
        title  year  fmovie_count
0  Kala Jigar  1939             1
1   Bindhaast  1999             1
2  Snegithiye  2000             1
3        Pihu  2018             1
Wall time: 1.11 s


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [13]:
%%time
# Write your sql query below

query = """
      SELECT a.year,a.fmovie_count,b.Total_count, a.fmovie_count*100.00/b.Total_count as Percentage FROM 
	(SELECT m.year, COUNT(*) AS fmovie_count FROM Movie m
		 WHERE m.MID NOT IN (
			  SELECT DISTINCT m.MID
			  FROM Movie m
			  INNER JOIN M_Cast mc ON m.MID = mc.MID
			  INNER JOIN Person p  ON mc.PID = p.PID
			  AND p.gender = 'Male'
			)
		AND m.MID IN (
			  SELECT DISTINCT m.MID
			  FROM Movie m
			  INNER JOIN M_Cast mc ON m.MID = mc.MID
			  INNER JOIN Person p  ON mc.PID = p.PID
			  AND p.gender = 'Female'
		)
		GROUP BY m.year) a,
	(SELECT m.year,COUNT(*) AS Total_count FROM Movie m GROUP BY m.year) b 
		
WHERE a.year = b.year
ORDER BY a.year
        
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
#q5b.head(6)
print(q5b)

(4, 4)
   year  fmovie_count  Total_count  Percentage
0  1939             1            2   50.000000
1  1999             1           66    1.515152
2  2000             1           64    1.562500
3  2018             1          104    0.961538
Wall time: 975 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [14]:
%%time
# Write your sql query below

query = """
SELECT title, COUNT(DISTINCT(PID)) cast_size
FROM M_Cast mc 
     INNER JOIN Movie m
              ON m.MID = mc.MID              
GROUP BY title ORDER BY cast_size DESC LIMIT 3
        
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
#q6.head(1)
print(q6)

(3, 2)
           title  cast_size
0  Ocean's Eight        238
1       Apaharan        233
2           Gold        215
Wall time: 166 ms


### Other Way

In [16]:
%%time
# Write your sql query below

query = """
SELECT title, COUNT(DISTINCT(Name)) cast_size
FROM M_Cast mc 
     INNER JOIN Person p 
              ON p.PID = mc.PID
     INNER JOIN Movie m
              ON m.MID = mc.MID              
GROUP BY title HAVING cast_size>100 ORDER BY cast_size DESC LIMIT 3
        
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
#q6.head(1)
print(q6)

(3, 2)
           title  cast_size
0  Ocean's Eight        238
1       Apaharan        233
2           Gold        214
Wall time: 536 ms


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [26]:
%%time
# Write your sql query below

query = """
SELECT d.year as dec_start,d.year + 9 AS dec_end, COUNT(*) AS movies_count
FROM
(SELECT DISTINCT year FROM Movie) d 
JOIN Movie m ON m.year>=d.year AND m.year<d.year + 10
GROUP BY d.year HAVING dec_end <= 2020
ORDER BY movies_count DESC   
        
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
#q7.head()
print(q7)

(71, 3)
   dec_start  dec_end  movies_count
0       2008     2017          1205
1       2009     2018          1202
2       2007     2016          1188
3       2005     2014          1170
4       2006     2015          1160
..       ...      ...           ...
66      1943     1952            25
67      1941     1950            14
68      1939     1948            11
69      1936     1945             7
70      1931     1940             6

[71 rows x 3 columns]
Wall time: 131 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [27]:
%%time
# Write your sql query below

query = """
SELECT f.num,f.director,f.actor,f.actor_name FROM
  (SELECT MAX(s.cnt) AS num,s.director,s.actor,s.act_name AS actor_name FROM
	(SELECT DISTINCT p.Name AS act_name,mc.PID AS actor, mc.MID,COUNT(mc.PID) AS cnt,d.PID as director FROM Person p 
		JOIN M_Cast mc ON mc.PID = p.PID 
		JOIN M_Director d ON d.MID = mc.MID
		GROUP BY d.PID,p.PID) AS s GROUP BY s.actor) AS f WHERE f.director = (SELECT PID FROM Person WHERE name = "Yash Chopra")
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head()

(228, 4)
Wall time: 1.8 s


,num,director,actor,actor_name
0,7,nm0007181,nm0004434,Shashi Kapoor
1,2,nm0007181,nm0007181,Yash Chopra
2,1,nm0007181,nm0015296,Akhtar-Ul-Iman
3,1,nm0007181,nm0019463,Murad Ali
4,1,nm0007181,nm0046230,Badri Prasad


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [28]:
%%time
# Write your sql query below

query = """
        
SELECT DISTINCT p2.Name,p2.PID FROM
Person p2 INNER JOIN M_Cast mc2 ON p2.PID = mc2.PID
INNER JOIN Movie m2 ON m2.MID = mc2.MID
WHERE p2.Name <> "Shah Rukh Khan" AND m2.MID IN
		(SELECT DISTINCT m1.MID FROM 
		Person p1 INNER JOIN M_Cast mc1 ON p1.PID = mc1.PID
		INNER JOIN Movie m1 ON m1.MID = mc1.MID
		WHERE p1.PID in (SELECT DISTINCT p.PID FROM 
						Person p INNER JOIN M_Cast mc ON p.PID = mc.PID
						INNER JOIN Movie m ON m.MID = mc.MID
						WHERE p.Name <> "Shah Rukh Khan" AND mc.MID IN 
								(SELECT mc.MID                      
								 FROM M_Cast mc 
						     		INNER JOIN Person p 
						              ON p.PID = mc.PID
						     		INNER JOIN Movie m
						              ON m.MID = mc.MID 
						 		WHERE p.name LIKE "%Shah%Rukh%Khan%"))) 
AND p2.PID Not in 
		(SELECT DISTINCT p1.PID FROM 
						Person p1 INNER JOIN M_Cast mc1 ON p1.PID = mc1.PID
						INNER JOIN Movie m1 ON m1.MID = mc1.MID
						WHERE p1.Name <> "Shah Rukh Khan" AND mc1.MID IN 
							 (SELECT mc.MID                      
								 FROM M_Cast mc 
						     		INNER JOIN Person p 
						              ON p.PID = mc.PID
						     		INNER JOIN Movie m
						              ON m.MID = mc.MID 
						 		WHERE p.name LIKE "%Shah%Rukh%Khan%"))
        
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(25698, 2)
Wall time: 2.91 s


,Name,PID
0,Freida Pinto,nm2951768
1,Rohan Chand,nm4575116
2,Damian Young,nm0949433
3,Waris Ahluwalia,nm1753302
4,Caroline Christl Long,nm6467532


__Considering bottom up approach, with level 1 or select1 from bottom__

__1.Select1 returns all the movies SRK acted in i.e 90 rows__

__2.Select2 returns all the actors(SRK1) who worked with SRK i.e distinct actor rows = 2382__

__3.Select3 returns all the movies SRK1 acted in i.e distinct movies rows = 3206__

__2ND part__

__4.Select4 returns all the actors(SRK2) acted in 3206 movies__

__5.p2.PID deselects all the common actors in SRK1 and SRK0 i.e 2382 + 1__

__6.Running complete code help us to get distinct actors who acted with SRK1 i.e 25698__